### Imports

In [79]:
# Imports

import pandas as pd
import numpy as np
from sklearn.cross_decomposition import CCA
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.multivariate.cancorr import CanCorr
import warnings
import seaborn as sns
import os
from functools import reduce
from country_list import countries_for_language
import pycountry_convert as pc

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.renderers.default = "notebook"

### Load and join raw data

Load and join raw data, both energy and social data.

- ***General comments***: From the webside "ourworldindata.org" we could see that data from 2020 was not completed so we discarded all data from 2020 to preserve accuracy of the data. 

In [80]:
# Path to raw data files
in_path = os.path.join(os.getcwd(), "../data/data_for_notebook")

#### Load and join energy consumption data

In [81]:
# Get energy consumption files
energy_data = [x for x in os.listdir(os.path.join(in_path,"EnergyData"))]

# Annual and per capita consumption paths
ac_paths = list(filter(lambda x: "ac" in x.lower(), energy_data))
pcc_paths = list(filter(lambda x: "pcc" in x.lower(), energy_data))

print(f"Merging ac_dfs ({len(ac_paths)})")
ac_dfs = [
    pd.read_csv(os.path.join(in_path, "EnergyData", filename))
    for filename in ac_paths
]
for df in ac_dfs:
    df.drop("Code", inplace=True, axis=1)
ac_final = reduce(
    lambda left, right: pd.merge(left, right, on=["Entity", "Year"], how="outer"),
    ac_dfs,
)

print(f"Merging pcc_dfs ({len(pcc_paths)})")
pcc_dfs = [
    pd.read_csv(os.path.join(in_path, "EnergyData", filename))
    for filename in pcc_paths
]
for df in pcc_dfs:
    df.drop("Code", inplace=True, axis=1)
pcc_final = reduce(
    lambda left, right: pd.merge(left, right, on=["Entity", "Year"], how="outer"),
    pcc_dfs,
)

# Annual consumption joined data
ac_joined = ac_final[(ac_final.Year >= 1965) & (ac_final.Year < 2020)].reset_index().drop(columns="index")
# Per capita consumption joined data
pcc_joined = pcc_final[(pcc_final.Year >= 1965) & (pcc_final.Year < 2020)].reset_index().drop(columns="index")


Merging ac_dfs (12)
Merging pcc_dfs (11)


In [82]:
pcc_joined.head(5)

,Entity,Year,Coal per capita (kWh),Fossil Fuels per capita (kWh),Energy per capita (kWh),Low-carbon energy per capita (kWh),Gas per capita (kWh),Nuclear per capita (kWh),Oil per capita (kWh),Renewables per capita (kWh),Wind per capita (kWh),Solar per capita (kWh),Hydro per capita (kWh)
0,Africa,1965,1010.461,2062.718,2183.369,120.653,29.811,0.0,1022.446,0.120653,0.0,0.0,120.653
1,Africa,1966,984.359,2107.879,2239.127,131.249,32.505,0.0,1091.015,0.131249,0.0,0.0,131.249
2,Africa,1967,981.168,2067.899,2201.498,133.600,31.327,0.0,1055.404,0.133600,0.0,0.0,133.600
3,Africa,1968,994.882,2111.486,2263.843,152.359,30.957,0.0,1085.647,0.152359,0.0,0.0,152.359
4,Africa,1969,978.624,2086.611,2259.926,173.317,35.267,0.0,1072.720,0.173317,0.0,0.0,173.317


#### Load and join social data

In [101]:
np.quantile(energy_data['Coal per capita (kWh)'][energy_data['Coal per capita (kWh)']],0.02)

nan

In [83]:
# Get social data files
socio_data = [x for x in os.listdir(os.path.join(in_path, "Socio_eco_data"))]
socio_data = [x for x in socio_data if '.DS_Store' not in x]

print(f"Merging socio_dfs ({len(socio_data)})")
socio_dfs = [
    pd.read_csv(os.path.join(in_path, "Socio_eco_data", filename))
    for filename in socio_data
]

for df in socio_dfs:
    if "Code" in df.columns:
        df.drop("Code", inplace=True, axis=1)
    for col in df.columns:
        if "annotations" in col:
            df.drop(col, inplace=True, axis=1)
socio_data_final = reduce(
    lambda left, right: pd.merge(left, right, on=["Entity", "Year"], how="outer"),
    socio_dfs,
)

socio_data_joined = (
    socio_data_final.sort_values(["Entity", "Year"])
    .reset_index()
    .drop(columns="index")
)

Merging socio_dfs (12)


In [84]:
socio_data_joined.head(5)

,Entity,Year,"Literacy rates (World Bank, CIA World Factbook, and other sources)",military_expenditure,GDP per capita,"Mortality rate, under-5 (per 1,000 live births)",GDP,Population (historical estimates),Human Development Index (UNDP),Life expectancy,"Indicator:Prevalence of obesity among adults, BMI &GreaterEqual; 30 (crude estimate) (%) - Sex:Both sexes","Gross enrolment ratio, tertiary, both sexes (%)",Individuals using the Internet (% of population),Total tax revenue (% of GDP) (ICTD (2021))
0,Afghanistan,-10000,NaN,NaN,NaN,NaN,NaN,14737.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,-9000,NaN,NaN,NaN,NaN,NaN,20405.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,-8000,NaN,NaN,NaN,NaN,NaN,28253.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,-7000,NaN,NaN,NaN,NaN,NaN,39120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,-6000,NaN,NaN,NaN,NaN,NaN,54166.0,NaN,NaN,NaN,NaN,NaN,NaN


### Simple preprocessing of joined data

Changing column names of social data and remove unwanted columns.

- ***General comments***: From the above short outline of the social data dataframe we see that we had a multiple of columns from the raw data, which we did not wish to include in the analysis. These are therefore discarded and the remaining column names are furthermore changed to more suitable names.

#### Simple preprocessing of social data

In [85]:
# Column renaming and removal of unwanted columns from social data
df = socio_data_joined.copy()

print(f"Preprocessing data (Column renaming and extraction)")

cols = list(df.columns)
for indx, col in enumerate(cols):
    if "Mortality rate, under-5" in col:
        cols[indx] = "Child mortality rate (under 5 years - %)"
        df.loc[:, col] = (1.0 * df.loc[:, col]) / 1000.0
    elif "Human Dev" in col:
        cols[indx] = "HDI"
    elif "Access to basic drinking water" == col:
        cols[indx] = "Basic_Drinking_Water_Rate"
    elif "Access to basic sanitation services" == col:
        cols[indx] = "Basic_Sanitation_Services_Rate"
    elif "GDP per" in col:
        cols[indx] = "GDP per capita ($)"
    elif "Life expec" in col:
        cols[indx] = "Life expectancy (years)"
    elif "Entity" in col:
        cols[indx] = "Entity"
    elif "Year" in col:
        cols[indx] = "Year"
    elif "tertiary" in col:
        cols[indx] = "Tertiary education (%)"
    elif "Total tax revenue" in col:
        cols[indx] = "Tax revenue of total GDP (%)"
    elif "Individuals using the Internet" in col:
        cols[indx] = "Internet users (%)"
    elif "Population" in col:
        cols[indx] = "Population"
    else:
        cols[indx] = "to_drop"
        df.drop(columns=col, inplace=True)

df.columns = [col for col in cols if "to_drop" not in col]

# Adjust for wrong unit for HDI
df["HDI"] = pd.to_numeric(df["HDI"])
for indx in range(len(df["HDI"])):
    if df.loc[indx, "HDI"] > 1:
        df.loc[indx, "HDI"] = df.loc[indx, "HDI"] / 1000

# Remove data prior to 1965, since we do not have any energy measures prior to this year.
print(f"Removing all rows with year prior to 1965")
df["Year"] = pd.to_numeric(df["Year"])
df = df[df.Year >= 1965].reset_index().drop(columns="index")
df = df[df.Year < 2020].reset_index().drop(columns="index")


socio_data_joined = df

Preprocessing data (Column renaming and extraction)
Removing all rows with year prior to 1965


##### Fraction of nan values in data 

In [86]:
nan_frac_social = socio_data_joined.isnull().sum().sum()/(socio_data_joined.shape[0]*socio_data_joined.shape[1])
nan_frac_energy = pcc_joined.isnull().sum().sum()/(pcc_joined.shape[0]*pcc_joined.shape[1])

print(f'The fraction of nan values in the social data is currently: {nan_frac_social:.3f}')
print(f'The fraction of nan values in the energy data is currently: {nan_frac_energy:.3f}')

The fraction of nan values in the social data is currently: 0.368
The fraction of nan values in the energy data is currently: 0.424


### Interpolating, extrapolating and splitting data on country/area level

*As seen in the above section, the amount of nan-values in our data is quite excessive at this point (about 43% for both datasets). During this section we manage to reduce the amount of non-nan values of the social data by about 7 percent points.*

The main idea for this preprocessing step is to enrich our data, in order for us to better model and vizualize the data in a meaningfull way. We are aware that this introduces bias into the dataset, but we did feel like this was a better choice overall. Below is a short description of, how we did interpolate and extrapolate data from known data:


- **Interpolating data:** We have done a linear interpolation for all datapoints in-between two known values wihtin the same *entity/country*. This means, that if we had a known value for *HDI* for Japan in 1985, no values for 1986-1992, and a known value for 1993, then we do linear interpolation of the datapoints in the timerange 1986-1992 based on 1985 and 1993.

- **Extrapolating data:** Besides linear interpolation of data, we also did simple exterapolation of unknown datapoints. Specifically we choose to assume, that any given observed value would likely be somewhat the same 5 years into the future (and 5 years into the past). That is, we did constant extrapolation of unknown data. We are aware that linear extrapolation or auto-regressive extrapolation would have maybe yield a more fair approximation, but we still feel like the constant assumption was fair, given the few amount of years we did extrapolate.

- **Splitting data on country/area level:** The raw data included both area specific and country specific measures. In the last part of this preprocessing steps, we did split the data into a country specific and area specific dataframe and discarded the area specific data.

#### Interpolating data

##### Interpolation function

In [87]:
# Interpolation function 

def interpolate_data(df_joined: pd.DataFrame):
    print(f"Interpolating data (linear)")

    df = df_joined.copy()
    # Get shapes
    N = df.shape[0]
    cols_to_interp = df.columns[2:]

    # Replace strings with nan
    df.replace("nan", np.nan, inplace=True)

    # Force columns to be numeric (non entity columns)
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col])

    # Make pseudo-dataset for interpolating mask
    df_interp_mask = df.copy()
    df_interp_mask[cols_to_interp] = df_interp_mask[cols_to_interp] * 0
    df_interp_mask.replace(np.nan, 0, inplace=True)

    # Go through each row - for missing years add rows and interpolate if possible
    for i in range(1, N):
        year_diff = df.Year[i] - df.Year[i - 1]
        if (year_diff > 1) & (df.Entity[i] == df.Entity[i - 1]):
            new_rows = np.array(
                [
                    [df.Entity[i]] * (year_diff - 1),
                    np.arange(df.Year[i - 1] + 1, df.Year[i]),
                ]
            ).T

            new_mask = np.array(
                [
                    [df.Entity[i]] * (year_diff - 1),
                    np.arange(df.Year[i - 1] + 1, df.Year[i]),
                ]
            ).T

            for col in cols_to_interp:
                if (np.isnan(df.loc[i, col]) == False) & (
                    np.isnan(df.loc[i - 1, col]) == False
                ):
                    new_rows = np.concatenate(
                        [
                            new_rows,
                            np.linspace(
                                df.loc[i - 1, col], df.loc[i, col], year_diff + 1
                            )[1:-1].reshape((year_diff - 1, 1)),
                        ],
                        axis=1,
                    )
                    new_mask = np.concatenate(
                        [
                            new_mask,
                            np.repeat(1, year_diff - 1).reshape((year_diff - 1, 1)),
                        ],
                        axis=1,
                    )
                else:
                    new_rows = np.concatenate(
                        [
                            new_rows,
                            np.array(np.repeat(np.nan, year_diff - 1)).reshape(
                                (year_diff - 1, 1)
                            ),
                        ],
                        axis=1,
                    )
                    new_mask = np.concatenate(
                        [
                            new_mask,
                            np.repeat(0, year_diff - 1).reshape((year_diff - 1, 1)),
                        ],
                        axis=1,
                    )

            new_rows = pd.DataFrame(new_rows, columns=df.columns)
            new_rows.replace("nan", np.nan, inplace=True)
            for col in new_rows.columns[1:]:
                new_rows[col] = pd.to_numeric(new_rows[col])

            new_mask = pd.DataFrame(new_mask, columns=df.columns)
            for col in new_mask.columns[1:]:
                new_mask[col] = pd.to_numeric(new_mask[col])

            df = (
                pd.concat([df.iloc[:i], new_rows, df.iloc[i:]], axis=0)
                .reset_index()
                .drop(columns="index")
            )
            df_interp_mask = (
                pd.concat(
                    [df_interp_mask.iloc[:i], new_mask, df_interp_mask.iloc[i:]], axis=0
                )
                .reset_index()
                .drop(columns="index")
            )
            N = N + year_diff - 1

    # Go through each column and interpolate values if possible
    for col in cols_to_interp:
        for i in range(N - 2):
            if not np.isnan(df.loc[i, col]):
                indx_old_non_nan = i
                while (df.Entity[i + 1] == df.Entity[i]) & (
                    np.isnan(df.loc[i + 1, col])
                ):
                    i = i + 1
                    if i == N - 1:
                        break
                if i == N - 1:
                    break
                if (df.Entity[i + 1] == df.Entity[i]) & (
                    indx_old_non_nan != i
                ):  # Non nan value followed by x nan values then non nan value (interpolation possible)
                    i = i + 1
                    df.loc[(indx_old_non_nan + 1) : (i - 1), col] = np.linspace(
                        df.loc[indx_old_non_nan, col],
                        df.loc[i, col],
                        i - indx_old_non_nan + 1,
                    )[1:-1]
                    df_interp_mask.loc[
                        (indx_old_non_nan + 1) : (i - 1), col
                    ] = np.repeat(1, i - indx_old_non_nan - 1)
                else:
                    i = i + 1
                    continue


    return df, df_interp_mask

##### Interpolate energy and social data

In [88]:
socio_data_inter, socio_data_inter_mask = interpolate_data(socio_data_joined)
pcc_inter, pcc_inter_mask = interpolate_data(pcc_joined)

print(f'Number of non-null values in raw joined data (energy pc): {(pcc_joined.isnull()==False).sum().sum()}')
print(f'Number of non-null values in interpolated data (energy pc): {(pcc_inter.isnull()==False).sum().sum()}')
print(f'Number of non-null values in raw joined data (social): {(socio_data_joined.isnull()==False).sum().sum()}')
print(f'Number of non-null values in interpolated data (social): {(socio_data_inter.isnull()==False).sum().sum()}')


Interpolating data (linear)
Interpolating data (linear)
Number of non-null values in raw joined data (energy pc): 71514
Number of non-null values in interpolated data (energy pc): 71568
Number of non-null values in raw joined data (social): 104201
Number of non-null values in interpolated data (social): 108376


**We see an improvement in the amount of data, especially for the social data. This will surely help us build a more stable ML model (although more biased), and more smooth visualizations.**

#### Extrapolating data

##### Extrapolation function

In [89]:
# Extrapolation function

def extrapolate_data(df_inter: pd.DataFrame, df_inter_mask: pd.DataFrame, x_extrap: int):
    df = df_inter.copy()
    df_extrap_mask = df_inter_mask.copy()
    
    print(f"Extrapolating data (max {x_extrap} years)")

    # Get shapes
    N = df.shape[0]
    cols_to_extrap = df.columns[2:]

    if x_extrap >= 1:
        for col in cols_to_extrap:
            for i in range(1, N):
                if (
                    np.isnan(df.loc[i - 1, col])
                    & (not np.isnan(df.loc[i, col]))
                    & (df.loc[i - 1, "Entity"] == df.loc[i, "Entity"])
                ):
                    m = 1
                    if i - m - 1 >= 0:
                        while (
                            np.isnan(df.loc[i - m - 1, col])
                            & (df.loc[i - m - 1, "Entity"] == df.loc[i, "Entity"])
                            & (m + 1 <= x_extrap)
                        ):  # Go back a max of x_extrap years
                            m = m + 1
                            if i - m == 0:
                                break

                    df.loc[i - m : i - 1, col] = np.repeat(df.loc[i, col], m)
                    df_extrap_mask.loc[i - m : i - 1, col] = np.repeat(2, m)

            i = 0
            while i < N - 1:
                if (
                    np.isnan(df.loc[i + 1, col])
                    & (not np.isnan(df.loc[i, col]))
                    & (df.loc[i + 1, "Entity"] == df.loc[i, "Entity"])
                ):
                    m = 1
                    if i + m + 1 <= N - 1:
                        while (
                            np.isnan(df.loc[i + m + 1, col])
                            & (df.loc[i + m + 1, "Entity"] == df.loc[i, "Entity"])
                            & (m + 1 <= x_extrap)
                        ):  # Go forward a max of x_extrap years
                            m = m + 1
                            if i + m == N - 1:
                                break
                    df.loc[i + 1 : i + m, col] = np.repeat(df.loc[i, col], m)
                    df_extrap_mask.loc[i + 1 : i + m, col] = np.repeat(2, m)
                    i += m + 1
                else:
                    i += 1

    return df, df_extrap_mask

##### Extrapolating energy and social data

In [90]:
socio_data_extrap, socio_data_extrap_mask = extrapolate_data(socio_data_inter, socio_data_inter_mask, 5)
pcc_extrap, pcc_extrap_mask = extrapolate_data(pcc_inter, pcc_inter_mask, 5)

print(f'Number of non-null values in interpolated data (energy pc): {(pcc_inter.isnull()==False).sum().sum()}')
print(f'Number of non-null values in extrapolated data (energy pc): {(pcc_extrap.isnull()==False).sum().sum()}')
print(f'Number of non-null values in interpolated data (social): {(socio_data_inter.isnull()==False).sum().sum()}')
print(f'Number of non-null values in extrapolated data (social): {(socio_data_extrap.isnull()==False).sum().sum()}')


Extrapolating data (max 5 years)
Extrapolating data (max 5 years)
Number of non-null values in interpolated data (energy pc): 71568
Number of non-null values in extrapolated data (energy pc): 71568
Number of non-null values in interpolated data (social): 108376
Number of non-null values in extrapolated data (social): 114765


**We also see an improvement in the amount of data here, but only for the social data.**

#### Fraction of nan values in data 

In [91]:
nan_frac_social = socio_data_extrap.isnull().sum().sum()/(socio_data_extrap.shape[0]*socio_data_extrap.shape[1])
nan_frac_energy = pcc_extrap.isnull().sum().sum()/(pcc_extrap.shape[0]*pcc_extrap.shape[1])

print(f'The fraction of nan values in the social data is currently: {nan_frac_social:.3f}')
print(f'The fraction of nan values in the energy data is currently: {nan_frac_energy:.3f}')

The fraction of nan values in the social data is currently: 0.320
The fraction of nan values in the energy data is currently: 0.425


For the social data we can see that we have manage to reduce the amount of nan-values with about 7 percent points!

**NB:** Be aware that it acutally seems like the fraction of nan-values for the energy data has become larger (which is also true), but the reason being is, that we were able to interpolate one of the columns in the energy data for a certain number of years, which leads to more nan values in the remaining columns (i.e. we add more rows to the data, with only one column being "filled" up with non-nan values.)

#### Splitting data on country/area level

##### Splitting function

In [92]:
# Splitting data on country and area level

def split_data_country_area(df_extrap: pd.DataFrame, df_extrap_mask: pd.DataFrame):
    df = df_extrap.copy()
    df_mask = df_extrap_mask.copy()

    print(f"Splitting data into countries and areas")

    entities = pd.Series(df.Entity.unique(), dtype="string")
    entities.replace({"&": "and"}, inplace=True, regex=True)
    countries = pd.Series(np.array(countries_for_language("en"))[:, 1], dtype="string")
    countries.replace({"&": "and"}, inplace=True, regex=True)
    countries.replace({"Congo - Brazzaville": "Congo"}, inplace=True, regex=True)
    countries.replace(
        {"Congo - Kinshasa": "Democratic Republic of Congo"}, inplace=True, regex=True
    )
    countries.replace({"Côte": "Cote"}, inplace=True, regex=True)
    countries.replace({"Curaçao": "Curacao"}, inplace=True, regex=True)
    countries.replace({"Czechoslovakia": "Czechia"}, inplace=True, regex=True)
    countries.replace({"Faroe Islands": "Faeroe Islands"}, inplace=True, regex=True)
    countries.replace({"Hong Kong SAR China": "Hong Kong"}, inplace=True, regex=True)
    # countries.replace({"Micronesia":"Micronesia (country)"},inplace=True, regex=True)
    countries.replace({"Macao SAR China": "Macao"}, inplace=True, regex=True)
    countries.replace({"Myanmar \(Burma\)": "Myanmar"}, inplace=True, regex=True)
    countries.replace(
        {"Palestinian Territories": "Palestine"}, inplace=True, regex=True
    )
    countries.replace(
        {"São Tomé and Príncipe": "Sao Tome and Principe"}, inplace=True, regex=True
    )
    countries.replace({"St\.": "Saint"}, inplace=True, regex=True)
    countries.replace(
        {"Saint Vincent and Grenadines": "Saint Vincent and the Grenadines"},
        inplace=True,
        regex=True,
    )
    countries.replace(
        {"Saint Martin": "Saint Martin (French part)"}, inplace=True, regex=True
    )
    countries.replace({"Timor-Leste": "Timor"}, inplace=True, regex=True)
    countries.replace(
        {"Saint Barthélemy": "Saint Barthelemy"}, inplace=True, regex=True
    )
    countries.replace(
        {"U\.S\. Virgin Islands": "United States Virgin Islands"},
        inplace=True,
        regex=True,
    )
    countries.replace({"Vatican City": "Vatican"}, inplace=True, regex=True)
    # countries.replace({"Wallis and Futuna":"Wallis and Futuna Islands"},inplace=True, regex=True)

    idx_country = []
    for ent in entities:
        if any(ent == country for country in countries):
            idx_country.append(True)
        else:
            idx_country.append(False)
    map_country = dict(zip(entities, idx_country))

    df_ent = df.Entity.astype("string").replace({"&": "and"}, regex=True)

    df_country = (
        df.iloc[np.array(df_ent.map(map_country))].reset_index().drop(columns="index")
    )
    df_area = (
        df.iloc[np.array(df_ent.map(map_country) == False)]
        .reset_index()
        .drop(columns="index")
    )
    df_mask_country = (
        df_mask.iloc[np.array(df_ent.map(map_country))]
        .reset_index()
        .drop(columns="index")
    )
    df_mask_area = (
        df_mask.iloc[np.array(df_ent.map(map_country) == False)]
        .reset_index()
        .drop(columns="index")
    )

    # Add continent info and remove countries with no continent info
    countries_code = pd.Series(np.array(countries_for_language('en'))[:,0],dtype="string")
    df_country['Continent'] = np.nan
    for indx, ent in enumerate(df_country['Entity']):
        try:
            cont = pc.convert_continent_code_to_continent_name(
            pc.country_alpha2_to_continent_code(countries_code.iloc[list(countries).index(ent)]))
            df_country.loc[indx,'Continent'] = cont
        except:
            continue;

    cols = df_country.columns[[0,-1]+list(np.arange(1,len(df_country.columns)-1,1))]
    df_country = df_country[cols]
    df_country = df_country[df_country['Continent'].notna()]


    return df_country, df_mask_country, df_area, df_mask_area

In [93]:
social_data, _, _, _ = split_data_country_area(socio_data_extrap, socio_data_extrap_mask)
energy_data, _, _, _ = split_data_country_area(pcc_extrap, pcc_extrap_mask)

Splitting data into countries and areas
Splitting data into countries and areas


### Comparing preprocessed data with raw data

In [94]:
socio_data_joined.describe()

,Year,GDP per capita ($),Child mortality rate (under 5 years - %),Population,HDI,Life expectancy (years),Tertiary education (%),Internet users (%),Tax revenue of total GDP (%)
count,16490.000000,8950.000000,11362.000000,1.335200e+04,5001.000000,13365.000000,6391.000000,7089.000000,5711.000000
mean,1993.090176,11207.043809,0.007053,6.776480e+07,0.651761,66.244199,22.774782,20.969479,19.895079
std,15.612882,13223.970673,0.007077,4.358233e+08,0.167121,10.674423,22.214878,27.488015,11.149605
min,1965.000000,377.580109,0.000170,6.450000e+02,0.190000,18.907000,0.000000,0.000000,0.085807
25%,1980.000000,2256.863037,0.001700,2.745148e+05,0.524000,59.675000,4.862370,0.158150,11.215454
50%,1994.000000,6540.000000,0.004380,3.929306e+06,0.677000,69.171000,16.174170,6.000000,17.591970
75%,2007.000000,14692.250000,0.010260,1.457919e+07,0.781000,74.080000,33.222464,36.000000,27.511119
max,2019.000000,156299.000000,0.040650,7.713468e+09,0.953000,86.751000,119.778748,99.701504,60.946429


In [95]:
social_data.describe()

,Year,GDP per capita ($),Child mortality rate (under 5 years - %),Population,HDI,Life expectancy (years),Tertiary education (%),Internet users (%),Tax revenue of total GDP (%)
count,12424.000000,8943.000000,10040.000000,1.242300e+04,7081.000000,12375.000000,8671.000000,7170.000000,6858.000000
mean,1992.004829,11369.042373,0.007161,2.420883e+07,0.633879,66.206943,21.160397,19.603739,19.558610
std,15.867833,13449.493058,0.007341,1.040754e+08,0.170723,10.712561,22.302553,27.731953,11.063812
min,1965.000000,377.580109,0.000170,1.126000e+03,0.190000,18.907000,0.000000,0.000000,0.085807
25%,1978.000000,2286.692383,0.001660,3.109885e+05,0.503000,59.548000,3.553769,0.000000,10.814146
50%,1992.000000,6655.000000,0.004180,3.985107e+06,0.663000,69.162000,13.641500,3.220229,17.444052
75%,2006.000000,14806.544922,0.010523,1.312171e+07,0.762000,74.054500,30.537685,32.896521,26.907533
max,2019.000000,156299.000000,0.040650,1.433784e+09,0.953000,86.751000,119.778748,99.701504,60.946429


In [24]:
energy_data.describe()

,Year,Coal per capita (kWh),Fossil Fuels per capita (kWh),Energy per capita (kWh),Low-carbon energy per capita (kWh),Gas per capita (kWh),Nuclear per capita (kWh),Oil per capita (kWh),Renewables per capita (kWh),Wind per capita (kWh),Solar per capita (kWh),Hydro per capita (kWh)
count,9141.000000,4064.000000,4064.000000,9141.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000,4064.000000
mean,1996.920578,5824.752946,32815.990878,25771.054994,5687.998717,10355.202020,1448.488227,16636.035918,4.239510,132.856207,29.120717,3686.924950
std,13.913575,9044.744974,34629.966122,36973.774913,13430.180511,23307.647484,3439.782146,17160.420317,12.581882,519.133407,128.804465,11016.169269
min,1965.000000,0.000000,124.143000,0.000000,0.000000,0.000000,0.000000,124.143000,0.000000,0.000000,0.000000,0.000000
25%,1986.000000,203.497250,11247.623500,2970.271000,179.755000,397.564750,0.000000,5316.907750,0.146810,0.000000,0.000000,73.557250
50%,1998.000000,2204.860000,25800.042500,13129.198000,1078.767500,4104.213000,0.000000,12319.816500,0.695548,0.000000,0.000000,541.506500
75%,2009.000000,8115.740500,40514.179750,35839.125000,4932.809250,10364.219500,429.328750,22516.859500,2.646389,3.079000,0.250250,2085.021250
max,2019.000000,96063.116000,308704.499000,651818.309000,146349.531000,278626.048000,24721.358000,151285.943000,146.349531,6928.511000,1797.682000,105566.894000


y = energy_data['Low-carbon energy per capita (kWh)']/energy_data['Energy per capita (kWh)']
np.corrcoef(social_data,y)